In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import filters, color, exposure
from skimage.feature import hog
from torchvision import models, transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision.datasets import CIFAR100, MNIST

In [ ]:
def blob_detection(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (256, 256))


    log_blobs = filters.gaussian(image, sigma=2) - filters.gaussian(image, sigma=4)


    dog_blobs = filters.difference_of_gaussians(image, 1, 4)


    fd, hog_image = hog(image, visualize=True)

    fig, axes = plt.subplots(1, 4, figsize=(12, 6))
    axes[0].imshow(image, cmap='gray'); axes[0].set_title("Original")
    axes[1].imshow(log_blobs, cmap='gray'); axes[1].set_title("LoG")
    axes[2].imshow(dog_blobs, cmap='gray'); axes[2].set_title("DoG")
    axes[3].imshow(hog_image, cmap='gray'); axes[3].set_title("HoG")
    plt.show()

In [ ]:
def train_resnet(model, train_loader, test_loader, epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")


    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total:.2f}%')

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [ ]:
trainset = CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = CIFAR100(root='./data', train=False, download=True, transform=transform)
train_loader = data.DataLoader(trainset, batch_size=64, shuffle=True)
test_loader = data.DataLoader(testset, batch_size=64, shuffle=False)

resnet18 = models.resnet18(pretrained=True)
resnet18.fc = nn.Linear(resnet18.fc.in_features, 100)
train_resnet(resnet18, train_loader, test_loader)

resnet34 = models.resnet34(pretrained=True)
resnet34.fc = nn.Linear(resnet34.fc.in_features, 100)
train_resnet(resnet34, train_loader, test_loader)

100%|██████████| 169M/169M [00:03<00:00, 48.9MB/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 112MB/s]
